**Create function that returns Monte Carlo results at a given lipid content:**

In [1]:
from biorefineries.lipidcane import system
from biorefineries.lipidcane import set_lipid_fraction
from biosteam.evaluation import Model, Metric
from biosteam.evaluation.evaluation_tools import triang # Creates triangular distribution at +-10%
from biosteam import find, PowerUtility

flowsheet = find.flowsheet['lipidcane']
biorefinery = find('lipidcane_sys')
tea = biorefinery.TEA
ethanol = flowsheet('ethanol')
lipid_cane = flowsheet('lipid_cane')
ethanol_density = 2.9866 # kg/gallon
gasoline_energy = 33.41 # kWhr/gallon
ethanol_per_GGE = 1.5

# %% Create metrics

get_MFSP = lambda: tea.solve_price(ethanol) * ethanol_density / ethanol_per_GGE # USD / GGE
get_GGE_flow = lambda: ethanol.massnet / ethanol_per_GGE # kg / hr
get_excess_power = lambda: sum([i._power_utility.rate for i in biorefinery.units
                                if i._has_power_utility]) # kW
get_excess_power_per_GGE = lambda: get_excess_power() / get_GGE_flow() # kWhr/GGE
get_feedstock_price = lambda: tea.solve_price(lipid_cane) # USD / kg

metrics = (Metric('MFSP', get_MFSP, 'USD/GGE'),
           Metric('Net electricity', get_excess_power_per_GGE, 'kWhr/GGE'),
           Metric('Feedstock price', get_feedstock_price, 'USD/kr'))

# %% Create model and add most sensitive parameters:
# Ethanol price, Lipid cane price, Electricity price, Fermentation efficiency, and Boiler efficiency

model = Model(biorefinery, metrics)
param = model.parameter

# Ethanol price
@param(element=ethanol, units='USD/kg',
       distribution=triang(ethanol.price))
def set_ethanol_price(price):
    ethanol.price = price
    
# Lipid cane price
@param(element=lipid_cane, units='USD/kg',
       distribution=triang(lipid_cane.price))
def set_lipid_cane_price(price):
    lipid_cane.price = price

# Electricity price
@param(element=PowerUtility, units='USD/kWhr',
       distribution=triang(PowerUtility.price))
def set_electricity_price(price):
    PowerUtility.price = price

# Fermentation efficiency
fermentation = flowsheet('R301')
@param(element=fermentation,
       distribution=triang(fermentation.efficiency))
def set_fermentation_efficiency(efficiency):
    fermentation.efficiency= efficiency
    
# Boiler efficiency
BT = flowsheet('BT')
@param(element=BT,
       distribution=triang(BT.boiler_efficiency))
def set_boiler_efficiency(boiler_efficiency):
    BT.boiler_efficiency = boiler_efficiency

# Load samples
N_samples = 1000
rule = 'L'
samples = model.sample(N_samples, rule)
model.load_samples(samples)
    
# Function that returns Monte Carlo results
def uncertainty_results_at_lipid_fraction(lipid_fraction):
    set_lipid_fraction(lipid_fraction)
    model.evaluate()
    return model.table.copy()


Example use:

In [2]:
table = uncertainty_results_at_lipid_fraction(0.02)
table

Element  Stream-ethanol Stream-lipid cane    Power utility Fermentation-R301  \
Variable Price [USD/kg]    Price [USD/kg] Price [USD/kWhr]        Efficiency   
0                 0.824            0.0339           0.0617              0.88   
1                 0.811            0.0329           0.0648             0.847   
2                 0.803            0.0362           0.0642             0.832   
3                 0.756            0.0371           0.0639             0.882   
4                  0.76            0.0364           0.0682             0.887   
..                  ...               ...              ...               ...   
995               0.798            0.0329           0.0606             0.906   
996                0.77             0.034           0.0658              0.89   
997               0.807            0.0362           0.0621             0.906   
998               0.751            0.0348           0.0662             0.947   
999               0.861            0.0327           0.0685             0.933   

Element  Boiler turbogenerator-BT    Biorefinery                             \
Variable        Boiler efficiency MFSP [USD/GGE] Net electricity [kWhr/GGE]   
0                            0.83           1.94                      -3.03   
1                           0.785           1.98                      -2.61   
2                           0.772           2.12                      -2.52   
3                           0.856           2.03                       -3.4   
4                           0.796           1.99                       -2.5   
..                            ...            ...                        ...   
995                         0.817           1.87                      -2.66   
996                          0.81           1.92                      -2.69   
997                         0.724           1.98                      -1.37   
998                          0.84           1.83                      -2.71   
999                         0.765           1.81                      -1.79   

Element                            
Variable Feedstock price [USD/kr]  
0                          0.0245  
1                          0.0222  
2                          0.0208  
3                          0.0209  
4                          0.0214  
..                            ...  
995                        0.0239  
996                         0.022  
997                        0.0242  
998                        0.0235  
999                        0.0297  

[1000 rows x 8 columns]

**Spearman correlation between all parameters and all metrics:**

In [3]:
model.spearman()

,"(Biorefinery, MFSP [USD/GGE])","(Biorefinery, Net electricity [kWhr/GGE])","(Biorefinery, Feedstock price [USD/kr])"
"Stream-ethanol price (USD/kg) [0.71, 0.789, 0.868]",0.0153,-0.0157,0.747
"Stream-lipid cane price (USD/kg) [0.0311, 0.0345, 0.038]",0.5,0.066,0.0306
"Power utility price (USD/kWhr) [0.0585, 0.065, 0.0715]",-0.125,0.0358,0.104
"Fermentation-R301 efficiency [0.81, 0.9, 0.99]",-0.815,0.452,0.61
"Boiler turbogenerator-BT boiler efficiency [0.72, 0.8, 0.88]",-0.107,-0.888,0.0973


**Feedstock price at given IRR and Ethanol selling price:**

In [6]:
def calculate_feedstock_price(IRR, ethanol_price_per_gallon):
    tea.IRR = IRR
    ethanol.price = ethanol_price_per_gallon * ethanol_density
    return tea.solve_price(ethanol) # USD/kg

Example usage:

In [7]:
calculate_feedstock_price(0.15, 2.15)

0.7914965908414144

**For a more complete uncertainty for about 96 uncertain parameters:**

In [8]:
# %% Create model and all default parameters (plus a few more)

model_extended = Model(biorefinery, metrics)
param = model_extended.parameter
model_extended.load_default_parameters(lipid_cane) # THIS LOADS ALL 90 OR SO PARAMETERS

# Fermentation efficiency
fermentation = flowsheet('R301')
@param(element=fermentation,
       distribution=triang(fermentation.efficiency))
def set_fermentation_efficiency(efficiency):
    fermentation.efficiency= efficiency
    
# Boiler efficiency
BT = flowsheet('BT')
@param(element=BT,
       distribution=triang(BT.boiler_efficiency))
def set_boiler_efficiency(boiler_efficiency):
    BT.boiler_efficiency = boiler_efficiency

# Load samples
N_samples = 1000
rule = 'L'
samples = model_extended.sample(N_samples, rule)
model_extended.load_samples(samples)
    
# Function that returns Monte Carlo results
def uncertainty_results_at_lipid_fraction_extended(lipid_fraction):
    set_lipid_fraction(lipid_fraction)
    model_extended.evaluate()
    return model.table.copy()


In [9]:
table = uncertainty_results_at_lipid_fraction_extended(0.02)
table

Element  Stream-ethanol Stream-lipid cane    Power utility Fermentation-R301  \
Variable Price [USD/kg]    Price [USD/kg] Price [USD/kWhr]        Efficiency   
0                 0.824            0.0339           0.0617              0.88   
1                 0.811            0.0329           0.0648             0.847   
2                 0.803            0.0362           0.0642             0.832   
3                 0.756            0.0371           0.0639             0.882   
4                  0.76            0.0364           0.0682             0.887   
..                  ...               ...              ...               ...   
995               0.798            0.0329           0.0606             0.906   
996                0.77             0.034           0.0658              0.89   
997               0.807            0.0362           0.0621             0.906   
998               0.751            0.0348           0.0662             0.947   
999               0.861            0.0327           0.0685             0.933   

Element  Boiler turbogenerator-BT    Biorefinery                             \
Variable        Boiler efficiency MFSP [USD/GGE] Net electricity [kWhr/GGE]   
0                            0.83           1.94                      -3.03   
1                           0.785           1.98                      -2.61   
2                           0.772           2.12                      -2.52   
3                           0.856           2.03                       -3.4   
4                           0.796           1.99                       -2.5   
..                            ...            ...                        ...   
995                         0.817           1.87                      -2.66   
996                          0.81           1.92                      -2.69   
997                         0.724           1.98                      -1.37   
998                          0.84           1.83                      -2.71   
999                         0.765           1.81                      -1.79   

Element                            
Variable Feedstock price [USD/kr]  
0                          0.0245  
1                          0.0222  
2                          0.0208  
3                          0.0209  
4                          0.0214  
..                            ...  
995                        0.0239  
996                         0.022  
997                        0.0242  
998                        0.0235  
999                        0.0297  

[1000 rows x 8 columns]